In [ ]:
import pandas as pd
import spacy
import re
import tqdm
from collections import Counter


In [2]:
ner_nlp = spacy.load("uk_ner_web_trf_13class")

/Users/linndfors/study/diploma/new_venv/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/linndfors/study/diploma/new_venv/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/linndfors/study/diploma/new_venv/lib/python3.10/site-packages/spacy_transformers/layers/hf_shim.py:137: UserWarning: Error loading saved torch state_dict with strict=True, likely due to differences between 'transformers' versions. Attempting to load with strict=False as a fallback...

If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current 'transformers' and 'spacy-transforme

In [4]:
df = pd.read_csv("/Users/linndfors/study/diploma/ner_for_fem/src/MAIN_finetuned_aya_for_the_test_parallel_dataset.csv")

In [5]:
df.head()

,finetuned,references,inputs,double_finetuned
0,офіцерка,офіцерка,офіцер,офіцер
1,мільярдер,мільярдер,мільярдерка,мільярдерка
2,пакистанець,пакистанець,пакистанка,пакистанка
3,Статутні внески найбільшого розміру не задекла...,Статутні внески найбільшого розміру не задекла...,Статутні внески найбільшого розміру не задекла...,Статутні внески найбільшого розміру не задекла...
4,отоларингологиня,отоларингологиня,отоларинголог,отоларинголог


In [6]:
dict_df = pd.read_csv('/Users/linndfors/study/diploma/dict_uk/out/dict_corp_lt.txt', delimiter=' ', header=None, names=['word', 'lemma', 'grammar'])

In [7]:
def define_sex(name):

    filtered_df = dict_df[(dict_df['lemma'] == name) & (dict_df['word'] == name)]
    if not filtered_df.empty:
        grammar_pers = filtered_df['grammar'].iloc[0]
        if re.search(r'fname', grammar_pers):
            if re.search(r'\bf\b', grammar_pers):
                return "F"
            elif re.search(r'\bm\b', grammar_pers):
                return "M"
            else:
                return "U"
        elif re.search(r'lname', grammar_pers):
            return "L"
        else:
            return "U"
    else:
        # print("No matches found for:", name)
        return "U"
    # except Exception as e:
    #     # print("error", e)
    #     return "U"

In [9]:
res = {"L": [], "F": [], "M": []}

for x in tqdm.tqdm(df['finetuned']):
    doc = ner_nlp(x)

    person_entities = [ent.text for ent in doc.ents if ent.label_ == "PERS"]
    for pers in person_entities:
        for n in pers.split(" "):
            gen = define_sex(n)
            if gen in res.keys():
                res[gen].append(n)
                # print(n, gen)

100%|██████████| 523/523 [12:25<00:00,  1.43s/it]


In [17]:
def calculate_name_frequency(name_dict):
    stats = {}
    counter_gender = {'L': 0, 'M': 0 , 'F': 0}
    counter = 0
    
    for group, names in name_dict.items():
        name_count = Counter(names)
        counter += len(name_count)
        stats[group] = name_count
        for _, n in name_count.items():
            counter_gender[group] += n
    
    print("Total:", counter)
    print(counter_gender)
    
    return stats, counter, counter_gender

In [18]:
name_stat = calculate_name_frequency(res)

Total: 170
{'L': 136, 'M': 56, 'F': 81}


In [ ]:
def calculate_top_name_frequency(name_dict, top_n=5):
    stats = {}
    
    for group, names in name_dict.items():
        name_count = Counter(names)
        
        top_names = name_count.most_common(top_n)
        stats[group] = top_names
    
    return stats

In [14]:
top_name_statistics = calculate_top_name_frequency(res, top_n=5)

for group, stats in top_name_statistics.items():
    print(f"Top 10 statistics for {group}:")
    for name, count in stats:
        print(f"  {name}: {count}")
    print()

Top 10 statistics for L:
  Супруненко: 3
  Попова: 3
  Сердюк: 2
  Злочевська: 2
  Пономарьов: 2

Top 10 statistics for F:
  Наталія: 20
  Валентина: 13
  Олександра: 7
  Анатолія: 5
  Олена: 5

Top 10 statistics for M:
  Олександр: 10
  Микола: 6
  Сергій: 5
  Володимир: 4
  Ігор: 3



In [19]:
orig_res = {"L": [], "F": [], "M": []}

for x in tqdm.tqdm(df['inputs']):
    doc = ner_nlp(x)

    person_entities = [ent.text for ent in doc.ents if ent.label_ == "PERS"]
    for pers in person_entities:
        for n in pers.split(" "):
            gen = define_sex(n)
            if gen in orig_res.keys():
                orig_res[gen].append(n)
                # print(n, gen)

  0%|          | 0/523 [00:00<?, ?it/s]

100%|██████████| 523/523 [12:13<00:00,  1.40s/it]


In [22]:
name_stat = calculate_name_frequency(orig_res)

Total: 187
{'L': 138, 'M': 65, 'F': 72}


In [23]:
top_name_statistics = calculate_top_name_frequency(orig_res, top_n=5)

for group, stats in top_name_statistics.items():
    print(f"Top 5 statistics for {group}:")
    for name, count in stats:
        print(f"  {name}: {count}")
    print()

Top 5 statistics for L:
  Янукович: 4
  Іванющенко: 3
  Сердюк: 2
  Злочевський: 2
  Пономарьов: 2

Top 5 statistics for F:
  Марія: 6
  Олена: 6
  Ірина: 5
  Олександра: 5
  Оксана: 4

Top 5 statistics for M:
  Олександр: 6
  Сергій: 6
  Микола: 5
  Андрій: 4
  Іван: 4

